# SE446 - Week 1B: HDFS Fundamentals

**Course:** SE 446 - Big Data Analytics  
**Instructor:** Prof. Anis Koubaa  
**Session:** Week 1, Session B

---

## Learning Objectives

By the end of this session, you will be able to:
1. Explain HDFS architecture (NameNode, DataNode)
2. Understand data replication and fault tolerance
3. Compare file formats: CSV, JSON, Parquet
4. Perform basic data loading in Python (simulating HDFS concepts)

---

## 1. HDFS Architecture

**HDFS** (Hadoop Distributed File System) stores files across multiple machines.

### Key Components

| Component | Role | Analogy |
|-----------|------|--------|
| **NameNode** | Master node, stores metadata | Library catalog |
| **DataNode** | Worker nodes, store actual data | Library shelves |
| **Block** | Fixed-size chunk of file (128 MB default) | Book chapters |

### Architecture Diagram

```
┌─────────────────────────────────────────────────────────────┐
│                        HDFS CLUSTER                         │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  ┌───────────────────────────────────┐                     │
│  │           NameNode                │                     │
│  │  • File names and paths           │                     │
│  │  • Block locations                │                     │
│  │  • Replication info               │                     │
│  └───────────────────────────────────┘                     │
│           │           │           │                         │
│           ▼           ▼           ▼                         │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐                  │
│  │DataNode 1│  │DataNode 2│  │DataNode 3│                  │
│  │ Block A  │  │ Block A  │  │ Block B  │                  │
│  │ Block B  │  │ Block C  │  │ Block A  │  ← Replication   │
│  │ Block C  │  │ Block B  │  │ Block C  │                  │
│  └──────────┘  └──────────┘  └──────────┘                  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Task 1: HDFS Concepts Quiz (5 points)

Answer the following questions about HDFS architecture.

In [ ]:
# TASK 1: Fill in the correct answers about HDFS

hdfs_quiz = {
    "Which node stores metadata (file names, block locations)?": "_____",  # NameNode or DataNode?
    "Which node stores the actual data blocks?": "_____",                   # NameNode or DataNode?
    "Default block size in HDFS (in MB)?": _____,                            # 64, 128, or 256?
    "Default replication factor?": _____,                                    # 1, 2, or 3?
    "If NameNode fails, what happens to the cluster?": "_____"              # 'continues' or 'stops'?
}

print("HDFS Architecture Quiz")
print("=" * 50)
for question, answer in hdfs_quiz.items():
    print(f"Q: {question}")
    print(f"A: {answer}\n")

---

## 2. Data Replication

HDFS replicates each block across multiple DataNodes for **fault tolerance**.

### Example: Replication Factor = 3

```
Original File: report.csv (500 MB)
                    │
                    ▼
        ┌───────────┴───────────┐
        │     Split into        │
        │    4 blocks (128 MB)  │
        └───────────┬───────────┘
                    │
    ┌───────┬───────┼───────┬───────┐
    ▼       ▼       ▼       ▼       ▼
Block 1  Block 2  Block 3  Block 4
(128MB)  (128MB)  (128MB)  (116MB)
    │       │       │       │
    └───────┴───────┴───────┴───────┐
                                    │
                          Each block copied
                          to 3 DataNodes
```

**Total storage used:** 500 MB × 3 = 1.5 GB

### Task 2: Calculate HDFS Storage (5 points)

Calculate the actual storage required for files in HDFS.

In [ ]:
# TASK 2: Calculate HDFS storage requirements

def calculate_hdfs_storage(file_size_mb, block_size_mb=128, replication_factor=3):
    """
    Calculate total storage used in HDFS.
    
    Args:
        file_size_mb: Size of the original file in MB
        block_size_mb: Size of each block (default 128 MB)
        replication_factor: Number of copies (default 3)
    
    Returns:
        Tuple of (num_blocks, total_storage_mb)
    """
    import math
    
    # TODO: Calculate number of blocks (round up!)
    num_blocks = _____  # <-- FILL THIS: math.ceil(file_size_mb / block_size_mb)
    
    # TODO: Calculate total storage with replication
    total_storage_mb = _____  # <-- FILL THIS: file_size_mb * replication_factor
    
    return num_blocks, total_storage_mb

# Test with different files
test_files = [
    ("small_log.txt", 50),      # 50 MB file
    ("dataset.csv", 500),        # 500 MB file
    ("video.mp4", 2048),         # 2 GB file
]

print("HDFS Storage Calculator")
print("=" * 60)
print(f"{'File':<20} {'Size':<10} {'Blocks':<10} {'Total Storage':<15}")
print("-" * 60)

for filename, size_mb in test_files:
    blocks, total = calculate_hdfs_storage(size_mb)
    print(f"{filename:<20} {size_mb} MB{'':<4} {blocks:<10} {total} MB")

---

## 3. File Formats Comparison

Different file formats have different properties:

| Format | Type | Compression | Schema | Best For |
|--------|------|-------------|--------|----------|
| **CSV** | Row-based | None | No | Simple data exchange |
| **JSON** | Row-based | None | Self-describing | APIs, configs |
| **Parquet** | Column-based | Yes | Embedded | Analytics, Big Data |

### Why Parquet for Big Data?

1. **Columnar storage**: Read only needed columns
2. **Compression**: 10x smaller than CSV
3. **Schema embedded**: No need for external schema files
4. **Predicate pushdown**: Filter at storage level

In [ ]:
# Setup: Install required libraries (run once)
!pip install pandas pyarrow -q

In [ ]:
# Let's create sample data and compare file formats
import pandas as pd
import numpy as np
import os

# Create sample data (10,000 rows)
np.random.seed(42)
n_rows = 10000

data = {
    'id': range(n_rows),
    'name': [f'User_{i}' for i in range(n_rows)],
    'age': np.random.randint(18, 65, n_rows),
    'salary': np.random.randint(30000, 150000, n_rows),
    'department': np.random.choice(['Engineering', 'Sales', 'Marketing', 'HR'], n_rows)
}

df = pd.DataFrame(data)
print("Sample Data:")
print(df.head())
print(f"\nTotal rows: {len(df):,}")

### Task 3: Compare File Formats (10 points)

Save the data in different formats and compare their sizes.

In [ ]:
# TASK 3: Save data in different formats and compare sizes

# Save as CSV
df.to_csv('sample_data.csv', index=False)
csv_size = os.path.getsize('sample_data.csv')

# Save as JSON
df.to_json('sample_data.json', orient='records')
json_size = os.path.getsize('sample_data.json')

# TODO: Save as Parquet
# HINT: Use df.to_parquet('filename.parquet')
_____  # <-- FILL THIS: df.to_parquet('sample_data.parquet')
parquet_size = os.path.getsize('sample_data.parquet')

# Calculate compression ratios
print("\nFile Size Comparison")
print("=" * 50)
print(f"{'Format':<15} {'Size (KB)':<15} {'Ratio vs CSV':<15}")
print("-" * 50)
print(f"{'CSV':<15} {csv_size/1024:.2f} KB{'':<6} 1.00x")
print(f"{'JSON':<15} {json_size/1024:.2f} KB{'':<6} {json_size/csv_size:.2f}x")
print(f"{'Parquet':<15} {parquet_size/1024:.2f} KB{'':<6} {parquet_size/csv_size:.2f}x")

In [ ]:
# Bonus: Read only specific columns from Parquet (columnar benefit!)

# TODO: Read only 'name' and 'salary' columns from Parquet
# HINT: Use pd.read_parquet('file', columns=['col1', 'col2'])

df_subset = _____  # <-- FILL THIS: pd.read_parquet('sample_data.parquet', columns=['name', 'salary'])

print("Reading only 'name' and 'salary' columns:")
print(df_subset.head())
print(f"\nColumns loaded: {list(df_subset.columns)}")

---

## 4. Loading Real Data

Let's load a sample of the NYC Taxi dataset - a classic Big Data example.

In [ ]:
# Load NYC Taxi data sample
# Note: This is a small sample for learning purposes

url = "https://raw.githubusercontent.com/plotly/datasets/master/uber-rides-data1.csv"

taxi_df = pd.read_csv(url)

print("NYC Taxi/Uber Data Sample")
print("=" * 50)
print(f"Shape: {taxi_df.shape[0]:,} rows × {taxi_df.shape[1]} columns")
print(f"\nColumns: {list(taxi_df.columns)}")
print(f"\nFirst 5 rows:")
taxi_df.head()

### Task 4: Basic Data Exploration (5 points)

Explore the taxi dataset using pandas.

In [ ]:
# TASK 4: Basic data exploration

# TODO: Get the number of rows
num_rows = _____  # <-- FILL THIS: len(taxi_df) or taxi_df.shape[0]

# TODO: Get the number of columns
num_cols = _____  # <-- FILL THIS: taxi_df.shape[1]

# TODO: Get memory usage in MB
memory_mb = _____  # <-- FILL THIS: taxi_df.memory_usage(deep=True).sum() / (1024*1024)

print("Dataset Summary")
print("=" * 40)
print(f"Number of rows:     {num_rows:,}")
print(f"Number of columns:  {num_cols}")
print(f"Memory usage:       {memory_mb:.2f} MB")

---

## 5. HDFS Commands Reference

In a real Hadoop cluster, you would use these commands:

| Command | Description | Example |
|---------|-------------|---------|
| `hdfs dfs -ls /` | List files | `hdfs dfs -ls /user/data/` |
| `hdfs dfs -put` | Upload file | `hdfs dfs -put local.csv /data/` |
| `hdfs dfs -get` | Download file | `hdfs dfs -get /data/file.csv ./` |
| `hdfs dfs -mkdir` | Create directory | `hdfs dfs -mkdir /user/me/` |
| `hdfs dfs -rm` | Delete file | `hdfs dfs -rm /data/old.csv` |
| `hdfs dfs -cat` | View file content | `hdfs dfs -cat /data/file.txt` |

*Note: In this course, we simulate HDFS operations using Python/Colab. Real HDFS operations will be covered using Databricks in later weeks.*

---

## 📝 Reflection Questions (For ExamGPT)

Answer these questions in the ExamGPT quiz at the end of class:

**Q1:** What is the role of the NameNode in HDFS?

**Q2:** If we store a 500 MB file with replication factor 3, how much total storage is used?

**Q3:** Why is Parquet better than CSV for Big Data analytics? (Give 2 reasons)

**Q4:** Looking at your Task 3 results, what was the compression ratio of Parquet vs CSV?

---

## ✅ Summary

Today you learned:
- [x] HDFS architecture: NameNode and DataNodes
- [x] Data replication for fault tolerance
- [x] File format comparison: CSV vs JSON vs Parquet
- [x] Basic data loading with pandas
- [x] HDFS command reference

**Next Week:** MapReduce Programming

In [ ]:
# Cleanup: Remove temporary files
import os
for f in ['sample_data.csv', 'sample_data.json', 'sample_data.parquet']:
    if os.path.exists(f):
        os.remove(f)
print("✅ Temporary files cleaned up!")